# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [2]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [4]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_train.shape, df_test.shape

((21458, 25), (5112, 24))

In [5]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [6]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [7]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [8]:
# 방법 1: 최적
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept=True),
    random_state=123
)
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']

df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index=x.index, columns=X_imp)
)

X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_train['loading'] = df_train['loading'].fillna(df_train['loading'].mean())
df_test['loading'] = df_test['loading'].fillna(df_train['loading'].mean())

In [16]:
(
    df_train.isna().sum().sum(),
    df_test.isna().sum().sum()
)

(0, 0)

In [19]:
X_all = df_test.columns.tolist()

- loading의 log 변환 
- attribute_0, _1: 무시
- LR + SFS: \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'\]
- LinearDiscriminantAnalysis: fit ~ predict, fit ~ transform
- LR + PCA: ['loading'] ['measurement0 ~ 17'], n_components = 7
- RandomForestClassifier + GSCV:  {'max_depth': 7, 'min_samples_split': 512, 'n_estimators': 15}

# Kaggle형 풀이 단계

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

Step 4: 모델 개선 작업을 합니다.

In [21]:
from sklearn.model_selection import GroupKFold
gcv = GroupKFold(4)
for train_idx, valid_idx in gcv.split(df_train[X_all], df_train['failure'], groups=df_train['product_code']):
    df_cv_train, df_valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    print(df_cv_train['product_code'].unique(), df_valid['product_code'].unique())

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [22]:
# GroupKFold를 이용한 교차 검증을 해봅니다.
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GroupKFold

gcv = GroupKFold(4)
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
cross_val_score(clf_lr, df_train[X_all], df_train['failure'], scoring='roc_auc', cv=gcv, groups=df_train['product_code'])

array([0.58822089, 0.58492694, 0.58894173, 0.59538985])

In [25]:
# 교차검증시 train 얻어오는 루틴
from sklearn.model_selection import cross_validate
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], scoring='roc_auc', cv=gcv, groups=df_train['product_code'],
    return_train_score=True
)

{'fit_time': array([0.03039122, 0.01943898, 0.02000332, 0.02532625]),
 'score_time': array([0.00500512, 0.010005  , 0.01447773, 0.01003838]),
 'test_score': array([0.58822089, 0.58492694, 0.58894173, 0.59538985]),
 'train_score': array([0.59262299, 0.59350682, 0.59192443, 0.58956962])}